In [1]:
import sys
sys.path.insert(0, '/home/farhan/hamza/Coco/cocoapi-master/PythonAPI')
from pycocotools.coco import COCO
sys.path.insert(0, '/home/farhan/hamza/dreamai/')
import cv_model
import utils
import obj_utils
from dai_imports import*

%load_ext autoreload
%autoreload 2

In [2]:
dataDir='/home/farhan/hamza/Coco/'
train_name='train2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,train_name)
train = os.path.join(dataDir,train_name)

In [3]:
# def draw_im(im, ann, cat_dict):
#     ax = obj_utils.img_grid(im, figsize=(16,16))
#     for a in ann:
# #         b = bb_hw(b)
#         obj_utils.draw_rect(ax, a['bbox'])
#         obj_utils.draw_text(ax, a['bbox'][:2], cat_dict[a['category_id']]['name'], sz=16)

# def get_box_name(ann,cat_dict):
#     box = []
#     name = []
#     for a in ann:
# #         b = bb_hw(b)
#         box.append(obj_utils.hw_bb(a['bbox']))
#         name.append(cat_dict[a['category_id']]['name'])
#     return (box,name)

# def get_box_name_str(ann,cat_dict):
    
#     b,n = get_box_name(ann=ann,cat_dict=cat_dict)
#     n = ' '.join(n)
#     b2 = []
#     for x in b:
#         b2+=x
#     b2 = [str(x) for x in b2]
#     b2 = ' '.join(b2)
#     return b2,n

In [4]:
# # initialize COCO api for instance annotations
# coco=COCO(annFile)

# cats = coco.loadCats(coco.getCatIds())
# cat_dict = {c['id']:c for c in cats}
# imgs = coco.loadImgs(coco.getImgIds())
# anns = [coco.loadAnns(coco.getAnnIds(img['id'])) for img in imgs]
# good_idx = [i for i,a in enumerate(anns) if len(a) > 0]
# imgs = np.array(imgs)[good_idx]
# anns = np.array(anns)[good_idx]
# imgs_paths = [os.path.join(train,img['file_name']) for img in imgs]
# imgs_ids = [img['file_name'] for img in imgs]
# bboxs_names = np.array([get_box_name_str(ann,cat_dict) for ann in anns])
# bboxs = bboxs_names[:,0]
# names = bboxs_names[:,1]
# b_lens = np.array([len(b.split())//4 for b in bboxs])
# n_lens = np.array([len(n.split()) for n in names])
# mask = b_lens == n_lens
# bboxs = np.array(bboxs)[mask]
# names = np.array(names)[mask]
# imgs_ids = np.array(imgs_ids)[mask]
# df = pd.DataFrame({'img': imgs_ids,'bbox': bboxs, 'name': names},columns=['img','bbox','name'])
# df.to_csv('/home/farhan/hamza/Coco/dai_coco.csv',index=False)
# df.head()

In [5]:
# DP = data_processing.DataProcessor(dataDir,train_csv='dai_coco.csv',tr_name=train_name,setup_data=True)

# data_processing.save_obj(DP,'DP_object_coco.pkl')

In [6]:
DP = data_processing.load_obj('DP_object_coco.pkl')

In [7]:
data_dict = DP.data_dict

In [8]:
data_dict['data_dfs']['val'].head()

,img,bbox,name,one_hot
0,000000438560.jpg,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
1,000000079357.jpg,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
2,000000509388.jpg,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
3,000000077481.jpg,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
4,000000284648.jpg,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."


In [9]:
data_dict['data_dfs'][train_name].head()

,img,bbox,name,one_hot
0,000000396866.jpg,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
1,000000568330.jpg,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
2,000000049283.jpg,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
3,000000509490.jpg,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
4,000000287006.jpg,"[tensor(0.), tensor(0.), tensor(0.), tensor(0....","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."


In [10]:
image_size = (256,256)
bs = 32

In [11]:
sets,loaders,sizes = DP.get_data(data_dict, image_size, bs = bs)

In [12]:
sizes

{'train2017': 67936, 'val': 10190, 'test': 1799}

In [13]:
print_every = sizes[train_name]//3//bs
print_every

707

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = cv_model.CustomSSDObjectDetection(model_name='resnet50',one_cycle_factor=None,
                         optimizer_name='sgd',device=device,
                        best_model_file = 'best_custom_ssd_coco_resnet50_sgd.pth',
                         class_names=data_dict['class_names'],num_classes=data_dict['num_classes'],
                              image_size = image_size,dropout_p = 0.5,add_extra=False)

Set_transfer_model: self.Model set to resnet50
Grids: [8, 4, 2, 1]
resnet50: setting head: CustomSSD_MultiHead
Transfer: best accuracy = 0.000
Setting optim SGD


In [15]:
net.load_state_dict(torch.load('best_custom_ssd_coco_resnet50_sgd.pth'))

In [16]:
net.unfreeze()

In [ ]:
lr = net.find_lr(loaders[train_name],plot=True)
net.optimizer.param_groups[0]['lr'] = lr
net.optimizer.param_groups[0]['lr']

In [ ]:
# net.optimizer.param_groups[0]['lr'] = 0.0001
# net.optimizer.param_groups[0]['lr']

In [ ]:
net.fit(loaders[train_name],loaders['val'],epochs=70,print_every=print_every)

In [17]:
data_processing.save_obj(net,'best_obj_updated.pkl')

In [ ]:
net.show_nms(loader=loaders['test'],num=30,dp=DP,
             score_thresh=0.25,nms_overlap=0.4)

In [ ]:
DP2 = data_processing.load_obj('DP_object_pascal.pkl')

data_dict2 = DP2.data_dict

image_size2 = (224,224)
bs2 = 45

sets2,loaders2,sizes2 = DP2.get_data(data_dict2, image_size2, bs = bs2)

In [ ]:
# net.load_state_dict(torch.load('best_ssd.pth'))
# imgs = data_processing.load_obj('test_batch.pkl')
net.show_nms(loader=loaders2['test'],num=30,dp=DP2,
             score_thresh=0.26,nms_overlap=0.4)

In [ ]:
# net.load_state_dict(torch.load('best_ssd.pth'))
# imgs = data_processing.load_obj('test_batch.pkl')
net.show_nms(loader=loaders2['val'],num=30,dp=DP2,
             score_thresh=0.26,nms_overlap=0.3)

In [ ]:
# net.load_state_dict(torch.load('best_ssd.pth'))
# imgs = data_processing.load_obj('test_batch.pkl')
net.show_nms(loader=loaders2['JPEGImages'],num=30,dp=DP2,
             score_thresh=0.26,nms_overlap=0.25)